### 제1유형

제공된 데이터(data/mtcars.csv)의 qsec 칼럼을 최소-최대 척도(Min-Max Scale)로 변환한 후 0.5보다 큰 값을 가지는 레코드 수를 [제출 형식]에 맞춰 답안 작성 페이지에 입력하시오.

[제출 형식]
- 정수(integer)로 입력
    - (단, 소수점을 포함한 경우 소수점 첫째 자리에서 반올림하여 계산)
- 정수 답안만 입력

```python
import pandas as pd

df = pd.read_csv("data/mtcars.csv")

# (X - min) / (max - min)

min_qsec = df['qsec'].min()
max_qsec = df['qsec'].max()
df['qsec'] = (df['qsec'] - min_qsec) / (max_qsec - min_qsec)

cond1 = (df['qsec'] > 0.5)
print(len(df[cond1]))
```

---

### 제2유형

제공된 데이터는 백화점 고객이 1년간 상품을 구매한 속성 데이터이다. 제공된 학습용 데이터(data/customer_train.csv)를 이용하여 백화점 구매 고객의 성별을 예측하는 모델을 개발하고, 개발한 모델에 기반하여 평가용 데이터(data/customer_test.csv)에 적용하여 얻은 성별 예측 결과를 아래 [제출 형식]에 따라 CSV 파일로 생성하여 제출하시오.
- 예측 결과는 ROC-AUC 평가지표에 따라 평가함
- 성능이 우수한 예측 모델을 구축하기 위해서는 데이터 정제, Feature Engineering, 하이퍼 파라미터(hyper parameter) 최적화, 모델 비교 등이 필요할 수 있음. 다만, 과적합에 유의하여야 함.

[제출 형식]
- CSV 파일명 : result.csv (파일명에 디렉토리-폴더 지정 불가)
- 예측 성별 칼럼명 : pred
- 제출 칼럼 개수 : pred 칼럼 1개
- 평가용 데이터 개수와 예측 결과 데이터 개수 일치 : 2,482개

```python
import pandas as pd
import numpy as np

train = pd.read_csv("data/customer_train.csv")
x_test = pd.read_csv("data/customer_test.csv")

# EDA
x_train = train.drop(['성별'], axis = 1)
y_train = train['성별']

# print(x_train.isnull().sum())
# print(y_train.isnull().sum())
# print(x_test.isnull().sum())

# 결측치 대체
median = x_train['환불금액'].median()
x_train['환불금액'] = x_train['환불금액'].fillna(median)
x_test['환불금액'] = x_test['환불금액'].fillna(median)

# 원핫인코딩
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

x_test = x_test.reindex(columns = x_train.columns, fill_value = 0)

# 분리
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, stratify = y_train, random_state = 2023)

# 모델링
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 2023)
model.fit(x_train, y_train)

y_pred = model.predict(x_val)

# 평가
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_val, y_pred)
print(auc)

# 제출
y_result = model.predict(x_test)
pd.DataFrame({'pred' : y_result}).to_csv("result.csv", index = False)

# 제출 파일 확인
result = pd.read_csv("result.csv")
print(result)
```

---

## 제3유형

제공된 데이터(data/Titanic.csv)는 타이타닉호의 침몰 사건에서 생존한 승객 및 사망한 승객의 정보를 포함한 자료이다. 아래 데이터를 이용하여 생존 여부(Survived)를 예측하고자 한다. 각 문항의 답을 [제출 형식]에 맞춰 답안 작성 페이지에 입력하시오. (단, 벌점화(penalty)는 부여하지 않는다.)

1. Gender와 Survived 변수 간의 독립성 검정을 실시하였을 때, 카이제곱 통계량은? (반올림하여 소수 셋째 자리까지 계산)
2. Gender, SibSp, Parch, Fare를 독립변수로 사용하여 로지스틱 회귀모형을 실시하였을 때, Parch 변수의 계수값은? (반올림하여 소수 셋째 자리까지 계산)
3. 위 ②번 문제에서 추정된 로지스틱 회귀모형에서 SibSp 변수가 한 단위 증가할 때 생존할 오즈비(Odds ratio) 값은? (반올림하여 소수 셋째 자리까지 계산)

[제출 형식]
- 소수 넷째 자리에서 반올림하여 소수 셋째 자리까지만 계산

```python
import pandas as pd
import numpy as np

df = pd.read_csv("data/Titanic.csv")

# ①
table = pd.crosstab(df['Gender'], df['Survived'])

from scipy.stats import chi2_contingency
statistic, pvalue, dof, expected = chi2_contingency(table)

print(round(statistic, 3))

# ②
df['Gender'] = df['Gender'].map({'male' : 0, 'female' : 1})

x = df[['Gender', 'SibSp', 'Parch', 'Fare']]
y = df['Survived']

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty = 'none')
model.fit(x, y)

# print(model.coef_)
# print(model.intercept_)
print(np.round(model.coef_[0][2], 3))

# ③
result = np.exp(model.coef_[0][1])
print(np.round(result, 3))
```